In [2]:
import wandb
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist

In [ ]:
wandb.login()

In [ ]:
wandb.init(project='assignment-1', entity="ishaan_maheshwari-indian-institute-of-technology-madras" )

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()